### Theory

- $ y_i = 1$  if probability is $ P_i $
- $ y_i = 0$  if probability is $ 1-P_i $

For n such observations Likehood function is 
 $$
 L= \prod_{i=0}^n P_i^{y_i}(1-P_i)^{1-y_i}
 $$
 
 
 
 




In [27]:
from statsmodels.base.model import GenericLikelihoodModel
import statsmodels.api as sm
import random
import scipy.stats as stats
import numpy as np

#### Endog is array of m (no of informed neighbors)

In [28]:
exog = [2,3,4,1,2,1,2,3,4,1,2,3,4,5,1,2,3,4,5,2,1,1,3,2,2,2,3,4,5,6,7,8,2,4,7,8]

#### get informed is the function that generates corresponding value give m , p (innovation param) and q (immitation param)

In [29]:
def get_informed(m, q):
    """

    :param m: no of active neighbors
    :param p: innovation param
    :param q: immitation param
    :return:
    """
    p = 1 - pow((1 - q), m) # probability of adoption 
    u = random.uniform(0, 1)
    if u < p:
        return 1
    return 0

#### To create dummy (sample observed data) lets assume q = 0.2

In [30]:
p = 0.1
q = 0.2
print p
print q

0.1
0.2


#### create exog based on given m, p and q

In [31]:
endog = []
for each_m in exog:
    endog.append(get_informed(each_m, q))
print endog

[1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1]


In [33]:
print "exog: (value of m)", exog
print "endog: (adopted/ non adopted)", endog

exog: (value of m) [2, 3, 4, 1, 2, 1, 2, 3, 4, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 2, 1, 1, 3, 2, 2, 2, 3, 4, 5, 6, 7, 8, 2, 4, 7, 8]
endog: (adopted/ non adopted) [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1]


#### Now our  Aim is: 
- Given endog and exog (observed data) , can we estimate params p and q

In [34]:
exog = sm.add_constant(exog, prepend=True)
print exog

[[ 1.  2.]
 [ 1.  3.]
 [ 1.  4.]
 [ 1.  1.]
 [ 1.  2.]
 [ 1.  1.]
 [ 1.  2.]
 [ 1.  3.]
 [ 1.  4.]
 [ 1.  1.]
 [ 1.  2.]
 [ 1.  3.]
 [ 1.  4.]
 [ 1.  5.]
 [ 1.  1.]
 [ 1.  2.]
 [ 1.  3.]
 [ 1.  4.]
 [ 1.  5.]
 [ 1.  2.]
 [ 1.  1.]
 [ 1.  1.]
 [ 1.  3.]
 [ 1.  2.]
 [ 1.  2.]
 [ 1.  2.]
 [ 1.  3.]
 [ 1.  4.]
 [ 1.  5.]
 [ 1.  6.]
 [ 1.  7.]
 [ 1.  8.]
 [ 1.  2.]
 [ 1.  4.]
 [ 1.  7.]
 [ 1.  8.]]


In [35]:
# The loglike function is not correct yet 
class MyProbit(GenericLikelihoodModel):
    def loglike(self, params):
        exog = self.exog
        endog = self.endog
        p = params[0]
        q = params[1]
        r = 1 - (1 - p) * pow((1 - q), endog)
        return stats.norm.logcdf(r*np.dot(exog, params)).sum()

In [36]:
sm_probit_manual = MyProbit(endog, exog).fit()

Optimization terminated successfully.
         Current function value: -0.000000
         Iterations: 45
         Function evaluations: 112


In [37]:
sm_probit_manual.params

array([ 0.98946106,  7.80918884])